<a href="https://colab.research.google.com/github/treezy254/Data-Science-Prep/blob/master/home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

X_full = pd.read_csv('train.csv', index_col="Id")
X_test_full = pd.read_csv('test.csv', index_col='Id')

# y = X_full.SalePrice
# X_full.head()
# features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
# X = X_full[features].copy()
# X_test = X_test_full[features].copy()

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,  random_state=0)

In [56]:
X_train.head

<bound method NDFrame.head of       MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
Id                                                                            
619           20         90.0    11694            9            5       2007   
871           20         60.0     6600            5            5       1962   
93            30         80.0    13360            5            7       1921   
818           20          NaN    13265            8            5       2002   
303           20        118.0    13704            7            5       2001   
...          ...          ...      ...          ...          ...        ...   
764           60         82.0     9430            8            5       1999   
836           20         60.0     9600            4            7       1950   
1217          90         68.0     8930            6            5       1978   
560          120          NaN     3196            7            5       2003   
685           60      

## Preliminary Investigation

In [59]:
X_train.shape

missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [60]:
from sklearn.ensemble import RandomForestRegressor

model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='absolute_error', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=300, criterion='absolute_error', max_depth=10, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [62]:
# from sklearn.metrics import mean_absolute_error

# def score_model(model, X_t=X_train, X_v=X_valid, y_t= y_train, y_v=y_valid):
#   model.fit(X_t, y_t)
#   preds = model.predict(X_v)
#   return mean_absolute_error(y_v, preds)

# for i in range(0, len(models)):
#   mae = score_model(models[i])
#   print("Model %d MAE: %d" % (i+1, mae))

In [63]:
best_model = model_5

In [65]:
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators=300, criterion='absolute_error', max_depth=10, random_state=0)
  model.fit(X_train, y_train)
  preds = model.predict(X_valid)
  return mean_absolute_error(y_valid, preds)

## Dropping columns with missing values

In [66]:
col_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

reduced_X_train = X_train.drop(col_with_missing, axis=1)
reduced_X_valid = X_valid.drop(col_with_missing, axis=1)

## Imputation

In [71]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

model = RandomForestRegressor(n_estimators=100, criterion='absolute_error', max_depth=10, random_state=0)
model.fit(imputed_X_train, y_train)

RandomForestRegressor(criterion='absolute_error', max_depth=10, random_state=0)

In [72]:
print("MAE Imputation:")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE Imputation:
18017.300005707762


In [73]:
final_X_test = pd.DataFrame(my_imputer.transform(X_test))
preds_test = model.predict(final_X_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [70]:
output = pd.DataFrame(({'Id': X_test.index, 'SalePrice': preds_test}))
output.to_csv('submission2.csv', index=False)

In [74]:
mine = pd.read_csv('submission2.csv')
mine.head()

,Id,SalePrice
0,1461,122950.250000
1,1462,159904.958333
2,1463,181779.861667
3,1464,180106.956667
4,1465,190867.298333


In [75]:
y.head()

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64